In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-19 00:48:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-04-19 00:48:43 (11.4 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Filter data to retrieve all rows where total votes count => 20
df1 = df.filter(df["total_votes"] >= 20)
df1.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [6]:
# Filter data to show percentage of helpful_votes => 50%
df2 = df1.filter((df1["helpful_votes"]/df1["total_votes"]) >= 0.5)
df2.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [7]:
# Filter data to show Vine reviews
paid_df = df2.filter(df2["vine"] == 'Y')
paid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48771843|R3KKUSGFZWSUIY|B00YXO5UXG|     555176890|Turtle Beach - Ea...|     Video Games|          5|           56|         63|   Y|                N|Quality Chat Headset|Not every situati...| 2015-08-04|
|         US|   53080186|R10FO5UKKVZBK2|B00XO041RQ|     238654494|PDP AG7 True Wire...|     Video Games|          3|    

In [8]:
# Filter to show non-Vine reviews
unpaid_df = df2.filter(df2["vine"] == 'N')
unpaid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [9]:
# Vine counts & percentages
total_paid = paid_df.count()
five_star_paid = paid_df.filter(paid_df['star_rating'] == 5).count()
four_star_paid = paid_df.filter(paid_df['star_rating'] == 4).count()
three_star_paid = paid_df.filter(paid_df['star_rating'] == 3).count()
two_star_paid = paid_df.filter(paid_df['star_rating'] == 2).count()
one_star_paid = paid_df.filter(paid_df['star_rating'] == 1).count()
percent_five_paid = five_star_paid/total_paid
percent_four_paid = four_star_paid/total_paid 
percent_three_paid = three_star_paid/total_paid
percent_two_paid = two_star_paid/total_paid
percent_one_paid = one_star_paid/total_paid

In [10]:
# Non-Vine counts and percentages
total_unpaid = unpaid_df.count()
five_star_unpaid = unpaid_df.filter(unpaid_df['star_rating'] == 5).count()
four_star_unpaid = unpaid_df.filter(unpaid_df['star_rating'] == 4).count()
three_star_unpaid = unpaid_df.filter(unpaid_df['star_rating'] == 3).count()
two_star_unpaid = unpaid_df.filter(unpaid_df['star_rating'] == 2).count()
one_star_unpaid = unpaid_df.filter(unpaid_df['star_rating'] == 1).count()
percent_five_unpaid = five_star_unpaid/total_unpaid
percent_four_unpaid = four_star_unpaid/total_unpaid
percent_three_unpaid = three_star_unpaid/total_unpaid
percent_two_unpaid = two_star_unpaid/total_unpaid
percent_one_unpaid = one_star_unpaid/total_unpaid

In [11]:
print("Total number of Vine reviews: {0:.0f}".format(total_paid)+"\n"+
      "Number of 5-Star Vine reviews: {0:.0f}".format(five_star_paid)+"\n"+
      "Number of 4-Star Vine reviews: {0:.0f}".format(four_star_paid)+"\n"+
      "Number of 3-Star Vine reviews: {0:.0f}".format(three_star_paid)+"\n"+
      "Number of 2-Star Vine reviews: {0:.0f}".format(two_star_paid)+"\n"+
      "Number of 1-Star Vine reviews: {0:.0f}".format(one_star_paid)+"\n"+
      "Percentage 5-Star Vine reviews: {0:.2%}".format(percent_five_paid) +"\n"
      "Percentage 4-Star Vine reviews: {0:.2%}".format(percent_four_paid) +"\n"+
      "Percentage 3-Star Vine reviews: {0:.2%}".format(percent_three_paid) +"\n"+
      "Percentage 2-Star Vine reviews: {0:.2%}".format(percent_two_paid) +"\n"+
      "Percentage 1-Star Vine reviews: {0:.2%}".format(percent_one_paid))

Total number of Vine reviews: 94
Number of 5-Star Vine reviews: 48
Number of 4-Star Vine reviews: 24
Number of 3-Star Vine reviews: 16
Number of 2-Star Vine reviews: 5
Number of 1-Star Vine reviews: 1
Percentage 5-Star Vine reviews: 51.06%
Percentage 4-Star Vine reviews: 25.53%
Percentage 3-Star Vine reviews: 17.02%
Percentage 2-Star Vine reviews: 5.32%
Percentage 1-Star Vine reviews: 1.06%


In [12]:
print("Total number of Non-Vine reviews: {0:,.0f}".format(total_unpaid)+"\n"+
      "Number of 5-Star Non-Vine reviews: {0:,.0f}".format(five_star_unpaid)+"\n"+
      "Number of 4-Star Non-Vine reviews: {0:,.0f}".format(four_star_unpaid)+"\n"+
      "Number of 3-Star Non-Vine reviews: {0:,.0f}".format(three_star_unpaid)+"\n"+
      "Number of 2-Star Non-Vine reviews: {0:,.0f}".format(two_star_unpaid)+"\n"+
      "Number of 1-Star Non-Vine reviews: {0:,.0f}".format(one_star_unpaid)+"\n"+
      "Percentage 5-Star Non-Vine reviews: {0:.2%}".format(percent_five_unpaid)+"\n"+
      "Percentage 4-Star Non-Vine reviews: {0:.2%}".format(percent_four_unpaid)+"\n"+
      "Percentage 3-Star Non-Vine reviews: {0:.2%}".format(percent_three_unpaid)+"\n"+
      "Percentage 2-Star Non-Vine reviews: {0:.2%}".format(percent_two_unpaid)+"\n"+
      "Percentage 1-Star Non-Vine reviews: {0:.2%}".format(percent_one_unpaid))

Total number of Non-Vine reviews: 40,471
Number of 5-Star Non-Vine reviews: 15,663
Number of 4-Star Non-Vine reviews: 6,738
Number of 3-Star Non-Vine reviews: 4,379
Number of 2-Star Non-Vine reviews: 3,388
Number of 1-Star Non-Vine reviews: 10,303
Percentage 5-Star Non-Vine reviews: 38.70%
Percentage 4-Star Non-Vine reviews: 16.65%
Percentage 3-Star Non-Vine reviews: 10.82%
Percentage 2-Star Non-Vine reviews: 8.37%
Percentage 1-Star Non-Vine reviews: 25.46%


In [13]:
print("Total number of Vine reviews: {0:,.0f}".format(total_paid)+"\n"+
      "Number of 5-Star Vine reviews: {0:,.0f}".format(five_star_paid)+"\n"+
      "Percentage 5-Star Vine reviews: {0:.2%}".format(percent_five_paid))

Total number of Vine reviews: 94
Number of 5-Star Vine reviews: 48
Percentage 5-Star Vine reviews: 51.06%


In [14]:
print("Total number of Non-Vine reviews: {0:,.0f}".format(total_unpaid)+"\n"+
      "Number of 5-Star Non-Vine reviews: {0:,.0f}".format(five_star_unpaid)+"\n"+
      "Percentage 5-Star Non-Vine reviews: {0:.2%}".format(percent_five_unpaid))

Total number of Non-Vine reviews: 40,471
Number of 5-Star Non-Vine reviews: 15,663
Percentage 5-Star Non-Vine reviews: 38.70%
